In [28]:
# Routines implementing the variational couple cluster
# method using the LCU method and FPOAA

import numpy as np                          # for rank_1_projector and other custom matrices
import math
from collections import Counter

from projectq import MainEngine
from projectq.ops import H, Ry, Rx, X, Y, Z, R, Ph, All, Measure, ControlledGate
                                            # Ph for global phase
                                            # R is for phase gates
from projectq.ops._basics import SelfInverseGate
                                            # because they don't have a named identity
        
from projectq.meta import Dagger, Compute, Uncompute, Control

from projectq.ops import QubitOperator
# from fermilib.ops import FermionOperator
# from fermilib.transforms import jordan_wigner 
                                            # for Jordan-Wigner transform 
                                            # and fermionic Hamiltonians
class IdGate(SelfInverseGate):
    """ Identity gate class """
    def __str__(self):
        return "I"
    
    @property
    def matrix(self):
        return np.matrix([1, 0], [0,1])
    
#: Shortcut (instance of) :class:`IdGate`
I = Id = IdGate()

# the LCU 'V' map that prepares the control state
# take the list of 'm' coefts as input
# return the ceil(\log m) qubit register as output

def lcu_control_state_prep(eng, coefts, reg, dim):
    m = len(coefts)
    # dim = math.ceil(math.log(m,2)) 
    
    size = pow(2, dim)
    
    weight = np.sum(coefts)
    probs = np.zeros(size)
    probs[0:m] = [abs(x) for x in coefts]
    probs = (1.0/weight) * probs 
    # print("prob vec: {}".format(probs))
    
    # compute the rotation angles required for preparing
    # the LCU control state, apply conditional rotations
    # following the method of Grover & Rudolph (2002)
    
    # we need to perform bifurcations over dim rounds
    
    for i in range(1,dim+1):
        block_size = 2**(dim-i+1)
        # print("block_size: {}".format(block_size))
               
        # in each round, need to bifurcate 2^(round - 1) blocks
        num_blocks = 2**(i-1)
#        print("num_blocks: {}".format(num_blocks))
        target = np.zeros((2**i, 2**i))
        for j in range(0, num_blocks):
            # break loop if we've already crossed the last non-zero coefficient
            if (j*block_size > m-1):
                print("Broke from j={} because we crossed the last element\n".format(int(j)))
                break
            
            start = j*block_size
            end = start + block_size
#            print("start: {}, end: {}".format(int(start),int(end)))
            
            vec_j = probs[start : end]
#            print("block to be bifurcated: {}".format(vec_j))
            # break loop if singleton
            if (len(vec_j)<=1):
                print("Broke from j={} because we encountered a singleton\n".format(int(j)))
                break
            
            left_cond_prob_j = bifurcate(vec_j)
            # print("left cond prob of vec_j: {}".format(left_cond_prob_j))
            
            # perform a rotation with angle corresponding to sqrt of this prob
            f_j = math.sqrt(left_cond_prob_j)
            ang_j = math.acos(f_j)
            
            temp = np.binary_repr(j)
            temp = temp.zfill(i-1)        # padding with zeros to get fixed length bin
            with Compute(eng):
                for j in range(0,i-1):
                    if (int(temp[j])==0):
                        X | reg[j]
            
            with Control(eng, reg[0:i-1]):
                Ry(2*ang_j) | reg[i-1]
            Uncompute(eng)   
    return reg

# bifurcate vector into left and right 
# halves, and return the conditional
# probability of the left half
# will always expect len(vec) a power of 2
def bifurcate(vec):
    m = len(vec)
    tot = np.sum(vec)
    if (tot==0):
        print("some segment has probability 0")
        return 0
    
    left = 0.0
    for i in range (0, m//2):          # // = int division op
        left += vec[i]
        
    return left/tot


# lu_controlled_unitary takes a 
# ctrl + sys register and returns
# it after applying the lcu unitary list
# to sys, controlled on ctrl

# include check to make sure that the 
# matrices supplied are unitary

def lcu_controlled_unitary(eng, list_of_U, coefts, ctrl, sys, sys_dim):
    size = len(list_of_U)
    ctrl_dim = math.ceil(math.log(size, 2))
    
    for i in range(0,size):
        temp = np.binary_repr(i)
        temp = temp.zfill(ctrl_dim)        # pad with zeros for fixed length bin
        
        with Compute(eng):
            for j in range(0,ctrl_dim):
                if (int(temp[j])==0):
                    X | ctrl[j]
        
        with Control(eng, ctrl):
            if (coefts[i]<0):
                Ph(math.pi) | sys[j]   # apply global phase -1
            for j in range(0, sys_dim):
                if (list_of_U[i][j]==I):
                    continue
                list_of_U[i][j] | sys[j]
                
        Uncompute(eng)
        
def lcu(eng, list_of_unitaries, coefts, ctrl, sys, ctrl_dim, sys_dim):
        with Compute(eng):
            lcu_control_state_prep(eng, coefts, ctrl, ctrl_dim)
        lcu_controlled_unitary(eng, list_of_unitaries, coefts, ctrl, sys, sys_dim)
        Uncompute(eng)

def postselect(ctrl, ctrl_dim):
    for idx in range(0, ctrl_dim):
        Measure | ctrl[idx]
        if (bool(ctrl[idx])):
            return 0
    return 1

# calculate lcu success probability
def lcu_success_prob(eng, reg, sys_dim, dim):
    eng.flush()
    prob = 0
    sysmax = pow(2, sys_dim)
    for i in range(0, sysmax):
        bin_i = np.binary_repr(i)
        bin_i = bin_i.zfill(dim)        # padding with zeros to get fixed length bin
        prob += eng.backend.get_probability(bin_i, reg)
    return prob

# conditional phase for amplitude amplification
def cond_phase(eng, ctrl, sys, phase):
    with Compute(eng):
        All(X) | ctrl
    with Control(eng, ctrl):
        Ph(phase) | sys[0]                   # give global phase to any one sys qubit
    Uncompute(eng)

# Fixed Point Oblivious Amplitude Amplification (FPOAA)
# unitary input_map prepares some target state with success prob 1-p
# Recursive algorithm, takes recursion depth as input
def fpoaa(eng, ctrl, sys, ctrl_dim, input_map, depth):
    phi = math.pi/3.0
    
    gate_seq = fpoaa_string(depth)
    t = len(gate_seq)
    for i in range(0,t):
        if (string[t-1-i]=='W'):
            input_map(data_blob)
        elif (string[t-1-i]=='M'):
            with Dagger(eng):
                input_map(data_blob)
        elif (string[t-1-i]=='R'):
            ControlledGate(Ph(phi), ctrl_dim) | (ctrl, sys)
        elif (string[t-1-i]=='S'):
            ControlledGate(Ph(-1*phi), ctrl_dim) | (ctrl, sys) 

# classical hacks
# some string helper functions for FPOAA
# fpoaa_string computes as a classical 
# string the sequence of operators to be
# applied for depth n FPOAA 
# string_dagger takes a string representing
# an fpoaa block and returns its dagger
def fpoaa_string(depth):
    if(depth==1):
        return "WRMRW"
    else:
        V = fpoaa_string(depth-1)
        U = dagger(V)
        return V + 'R' + ''.join(U) + 'R' + V
    
def dagger(source):
    string = list(source)
    t = len(string)
    temp = [None] * t
    
    for i in range(0, t):
        if (string[t-1-i]=='W'):
             temp[i] = 'M'
        elif (string[t-1-i]=='M'):
             temp[i] = 'W'
        elif (string[t-1-i]=='R'):
             temp[i] = 'S'
        elif (string[t-1-i]=='S'):
            temp[i] = 'R'
    return temp


# code snippet for backend.cheat
def print_wavefunction(eng):
    eng.flush()
    print("The net backend state is \n")
    print(eng.backend.cheat())
    
# print bitstring probability amplitudes
def print_amplitudes(eng, reg, dim):
    eng.flush()
    maxout = pow(2, dim)
    for i in range(0, maxout):
        bin_i = np.binary_repr(i)
        bin_i = bin_i.zfill(dim)        # padding with zeros to get fixed length bin
        temp = eng.backend.get_amplitude(bin_i, reg)
        print("amplitude of string {} (i.e of {}) is {}".format(bin_i, int(i), temp))

# print bitstring probabilities
def print_probabilities(eng, reg, dim):
    eng.flush()
    maxout = pow(2, dim)
    for i in range(0, maxout):
        bin_i = np.binary_repr(i)
        bin_i = bin_i.zfill(dim)        # padding with zeros to get fixed length bin
        temp = eng.backend.get_probability(bin_i, reg)
        print("probability of string {} (i.e of {}) is {}".format(bin_i, int(i), temp))


# print output string after measurement
def print_mmt_output(reg, dim):
    eng.flush()
    # make sure mmt has been made: ideally use some flag
    # Just in case, mmt now - if already mmed, this causes no difference
    All(measure) | reg
    for i in range(0,dim):
        print("{}".format(int(reg[dim-1-i])), end=' ')
        
# testing root2 * H = X + Z
def test_hadamard1(eng, rounds=1):
    root2 = 1.0/math.sqrt(2)
    
    # H is a good test because X + Z = root2 * H
    
    sys_dim = 1
    A = [[X], [Z]]      
    coefts = [1/root2, 1/root2]
    
    m = len(coefts)
    ctrl_dim = math.ceil(math.log(m,2))
    dim = sys_dim + ctrl_dim
   
    num_1=0
    for i in range(0,rounds):
        ctrl = eng.allocate_qureg(ctrl_dim)
        sys = eng.allocate_qureg(sys_dim)
        lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
                
        success = postselect(ctrl, ctrl_dim)
        if (success):
            H | sys[0]     # try to uncompute the H applied to qubit 1
            Measure | sys[0]
            num_1 += int(sys[0])
        else:
            All(Measure) | ctrl + sys

        eng.flush()
#        print("val={}".format(int(anc[0])))

    All(Measure) | ctrl + sys
    eng.flush()
    print("num of 1 = {}".format(num_1))


# testing 2-qbit hadamard 2HH=(X+Z)(X+Z)
def test_hadamard2(eng, rounds=1):
    sys_dim = 2
       
    # newer versions of ProjectQ overload 
    # '|' for Pauli strings 
    A = [[X,X], [X,Z], [Z,X], [Z,Z]]
    coefts = [0.5, 0.5, 0.5, 0.5]
    
    m = len(coefts)
    ctrl_dim = math.ceil(math.log(m,2))
    dim = sys_dim + ctrl_dim
   
    num_1=0
    num = np.zeros(pow(2,sys_dim))
    for i in range(0,rounds):
        ctrl = eng.allocate_qureg(ctrl_dim)
        sys = eng.allocate_qureg(sys_dim)
        lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
        
        print("Amplitudes of ctrl+sys state:\n")
        print_amplitudes(eng, ctrl+sys, dim)

        success = postselect(ctrl, ctrl_dim)
        if (success):
#             All(H) | sys     # try to uncompute the HH applied to sys
            print("Amplitudes of ctrl+sys state after postselection:\n")
            print_amplitudes(eng, ctrl+sys, dim)
            All(Measure) | sys
           
            # repeating and measuring frequencies
            if (int(sys[0])==0):
                if (int(sys[1])==0):
                    num[0] += 1
                else:
                    num[1] = num[1] + 1
            else:
                if (int(sys[1])==0):
                    num[2] += 1
                else:
                    num[3] += 1
            num_1 += int(sys[0]) + int(sys[1])
        else:
            All(Measure) | ctrl + sys
        eng.flush()

    print("frequencies: ")
    for i in range(len(num)):
        print("{} ".format(int(num[i])))
    All(Measure) | ctrl + sys
    eng.flush()
    print("num of 1 = {}".format(num_1))


# one possibility: a family of 2-qubit unitaries
# controlled phase
# CP(1,2) = 0.5 * (\id + Z1 + Z2 - Z1 Z2 )
def test_controlled_phase_12(eng, oaa_rounds = 0, fpoaa_depth = 0):
    sys_dim = 2
       
    # newer versions of ProjectQ overload 
    # '|' for Pauli strings 
    A = [[I,I], [Z,I], [I,Z], [Z,Z]]
    coefts = [0.5, 0.5, 0.5, -0.5]
    
    m = len(coefts)
    ctrl_dim = math.ceil(math.log(m,2))
    dim = sys_dim + ctrl_dim
    
    ctrl = eng.allocate_qureg(ctrl_dim)
    sys = eng.allocate_qureg(sys_dim)
    
    # initialise sys state to the unif superposition
    All(H) | sys
#    print("Amplitudes of initial ctrl+sys state:\n")
#    print_amplitudes(eng, ctrl+sys, dim)
    
    lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
    
    print("Amplitudes of ctrl+sys state after lcu:\n")
    print_amplitudes(eng, ctrl+sys, dim)
    
    prob = lcu_success_prob(eng, ctrl+sys, sys_dim, dim)
    print("lcu success prob = {}\n".format(float(prob)))
    
    # testing amplitude amplification
    # case 1) FPOAA
    if (fpoaa_depth > 0):
        phi = math.pi/3.0
        gate_seq = fpoaa_string(fpoaa_depth)
        print("\n fpoaa gate_seq is {} \n".format(str(gate_seq)))
        t = len(gate_seq)
        # the rightmost operator is always W
        # which has already been applied above in the lcu step
        for i in range(1,t):
            if (gate_seq[t-1-i]=='W'):
                lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
            elif (gate_seq[t-1-i]=='M'):
                with Dagger(eng):
                    lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
            elif (gate_seq[t-1-i]=='R'):
                cond_phase(eng, ctrl, sys, phi)
            elif (gate_seq[t-1-i]=='S'):
                cond_phase(eng, ctrl, sys, -1*phi)
        prob = lcu_success_prob(eng, ctrl+sys, sys_dim, dim)
        print("lcu success prob after FPOAA({}) = {}\n".format(int(fpoaa_depth), float(prob)))
#         print("Probabilities of ctrl+sys state after lcu & FPOAA({}):\n".format(int(fpoaa_depth)))
#         print_probabilities(eng, ctrl+sys, dim)
        
    
    # case 2) OAA
    # repeat S = -(W R M R) oaa-1 times
    elif (oaa_rounds > 0):
        phi = -1*math.pi
        for i in range(0, oaa_rounds):
            cond_phase(eng, ctrl, sys, phi)
            with Dagger(eng):
                lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
            size = pow(2, ctrl_dim)
            for i in range(1,size):                # -R flips sign of everything except 00..0
                temp = np.binary_repr(i)
                temp = temp.zfill(ctrl_dim)        # pad with zeros for fixed length bin
            with Compute(eng):
                for j in range(0,ctrl_dim):
                    if (int(temp[j])==0):
                        X | ctrl[j]
            with Control(eng, ctrl):
                Ph(phi) | sys[0]                   # flip sign using any one sys qubit
            Uncompute(eng)
            lcu(eng, A, coefts, ctrl, sys, ctrl_dim, sys_dim)
        print("Amplitudes of ctrl+sys state after lcu & OAA({}):\n".format(int(oaa_rounds)))
        print_amplitudes(eng, ctrl+sys, dim)
    
    success = postselect(ctrl, ctrl_dim)
    if (success):
#       ControlledGate(R(math.pi), 1) | (sys[0], sys[1])     # try to uncompute the CPhase applied to sys
#       All(H) | sys                                         # uncompute the initial sys state
        print("Amplitudes of ctrl+sys state after postselection:\n")
        print_amplitudes(eng, ctrl+sys, dim)
        All(Measure) | sys
    else:
        All(Measure) | ctrl + sys
    eng.flush()

    All(Measure) | ctrl + sys
    eng.flush()



if __name__ == "__main__":
    eng = MainEngine()
    test_controlled_phase_12(eng, 2)

Amplitudes of ctrl+sys state after lcu:

amplitude of string 0000 (i.e of 0) is (0.24999999999990513-2.585288838878661e-14j)
amplitude of string 0001 (i.e of 1) is (0.24999999999999994+2.585288838878661e-14j)
amplitude of string 0010 (i.e of 2) is (0.24999999999999994+2.585288838878661e-14j)
amplitude of string 0011 (i.e of 3) is (-0.25000000000009476-2.585288838878661e-14j)
amplitude of string 0100 (i.e of 4) is (-0.2500000000000689-2.5852888388779472e-14j)
amplitude of string 0101 (i.e of 5) is (0.2500000000000258+2.5852888388779472e-14j)
amplitude of string 0110 (i.e of 6) is (-0.24999999999997408+2.5852888388779472e-14j)
amplitude of string 0111 (i.e of 7) is (-0.249999999999931-2.5852888388779472e-14j)
amplitude of string 1000 (i.e of 8) is (-0.2500000000000689-2.5852888388779475e-14j)
amplitude of string 1001 (i.e of 9) is (-0.24999999999997408+2.5852888388779475e-14j)
amplitude of string 1010 (i.e of 10) is (0.2500000000000258+2.5852888388779475e-14j)
amplitude of string 1011 (i

IndexError: list index out of range

# Implementing controlled operations |i><i|\otimes U_i using 'Control' from meta

Take any integer in its binary form, say 12 = 1100 and then do the right combination of X's to map 1100 ---> 1111, the target string being always the all 1s string. Then use Control. 


Strange issue : some parts of the lcu junk state are picking up phase factors of -1
For example, with CPhase, the post-lcu state should be
        00 ( 0+ + 1- )
       +01 ( 1+ + 0- )
       +10 ( 0+ - 1- )
       +11 ( 1+ - 0- )
but our code produces
        00 ( 0+ + 1- )
       -01 ( 1+ + 0- )
       -10 ( 0+ - 1- )
       +11 ( 1+ - 0- )
       
Is this an artifact of the use of X's in the control segments? Note that the negative sign seems to show up whenever there's an odd parity string.